In [ ]:
pip install pandas

In [ ]:
pip install TextBlob

In [1]:
# Bibliotecas

import json
import pandas as pd
import gzip
import ast
from datetime import datetime
import numpy as np
import csv
import time
import string
from textblob import TextBlob

In [2]:
 # Hago función para descomprimir el archivo en datos
 
 def abrir_archivo_comprimido(nombre_archivo=r"C:\Users\Usuario\Henry\PI1_MachineLearning\Consignas y Edas\user_reviews.json.gz"):  
   try:
      with gzip.open('user_reviews','rt', encoding='utf-8') as archivo_comprimido:
         datos=  json.load(archivo_comprimido)
                     
      return dataframe
       
       # Añado un bloque try y except para manejar excepciones como FileNotFoundError y otros errores que puedan ocurrir al abrir el archivo.

   except FileNotFoundError:
        print(f"El archivo '{user_reviews}' no se encontró.")
   except Exception   as e:
        print(f"Error al abrir el archivo comprimido: {str(e)}") 


In [3]:
# Desarrollo función leer_Json_gzip() que lee los archivos 'comprimidos.json.gz` y devuelve una lista de diccionarios Python.
def leer_json_gzip(user_reviews):

    # Abrir el archivo users_items.json.gz en modo de lectura y decodificamos el contenido del archivo.
    with gzip.open(user_reviews, 'rt', encoding='utf-8') as miArchivo:

        # Recorremos el contenido del archivo y evaluamos cada línea como una expresión Python.
        # El resultado de la evaluación es un diccionario Python.
        # La función strip() elimina los espacios en blanco al principio y al final de la línea.
        return [ast.literal_eval(line.strip()) for line in miArchivo]
    #Usamos ast.literal_eval para convertir cada linea en un diccionario y lo agregamos a la lista

ast.literal_eval es necesario porque los datos en el archivo parecen estar en formato JSON pero almacenados como cadenas en lugar de objetos de Python. literal_eval es más seguro que eval ya que solo evalúa expresiones literales de Python y evita la ejecución de código malicioso.

In [ ]:
#Como son cadenas debo recorrerlo para llenar el archivo

datos = []
for i  in gzip.open('user_reviews.json.gz'):
    datos.append(ast.literal_eval(i.decode('utf-8')))

df_user_reviews= pd.DataFrame(datos)

df_user_reviews

In [7]:
df_user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [8]:
# Dado que veo la columna reviews anidadada toca usar explode para desanidarla

df_user_reviews_exploded = df_user_reviews.explode('reviews')


In [14]:
df_user_reviews_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   59333 non-null  object
 1   user_url  59333 non-null  object
 2   reviews   59305 non-null  object
dtypes: object(3)
memory usage: 1.8+ MB


In [ ]:
df_user_reviews_exploded

In [11]:
df_user_reviews_desanidado = pd.json_normalize(df_user_reviews_exploded['reviews'])

In [13]:
df_user_reviews_desanidado

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
59328,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,,Posted July 20.,,730,No ratings yet,True,:D


En el archivo user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' 
aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral 
y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos 
de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1


In [15]:
# Se debe reindexar para que se organice nuevamente

df_user_reviews_exploded.reset_index(inplace=True)

In [16]:
#concateno el df exploded con el desanidado
df_user_reviews_sentimiento = pd.concat([df_user_reviews_exploded, df_user_reviews_desanidado], axis=1)

In [ ]:
df_user_reviews_sentimiento

In [19]:
# borro la columna reviews ya que al desanidarla veo que toda la información está en el df principial
df_user_reviews_sentimiento = df_user_reviews_sentimiento.drop('reviews',axis=1)

In [ ]:
df_user_reviews_sentimiento 

In [21]:
# De una vez borro columnas que no usare por que la información es confusa: index, funny, last_edited, helpul

Columas_borrar_sentimiento = ['index', 'funny', 'last_edited', 'helpful']

In [22]:
df_user_reviews_sentimiento = df_user_reviews_sentimiento.drop(Columas_borrar_sentimiento, axis=1)

In [ ]:
df_user_reviews_sentimiento 

En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' 
aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral 
y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos 
de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1

def analizar_sentimiento(texto):
    analysis = TextBlob(texto)
    # Devuelve la polaridad del sentimiento (-1 a 1)
    return analysis.sentiment.polarity

    Aplica la función a la columna relevante de tu DataFrame:
    df['Sentimiento'] = df['Texto'].apply(analizar_sentimiento)

In [ ]:
# La función textblob no pme corre por el tipo de datos por que la columna review tiene que ser string
tipos_de_datos = df_user_reviews_sentimiento.dtypes
print(tipos_de_datos)

In [25]:
# Cambio el tipo de review a string

df_user_reviews_sentimiento['review'] = df_user_reviews_sentimiento['review'].astype(str)

def analizar_sentimiento(texto):
    analysis = TextBlob(texto)
    # Devuelve la polaridad del sentimiento (-1 a 1)
    return analysis.sentiment.polarity

    Aplica la función a la columna relevante de tu DataFrame:
    df['Sentimiento'] = df['Texto'].apply(analizar_sentimiento)

In [26]:
def analisis_sentimiento(Texto):
    analisis = TextBlob(Texto)
    if analisis.polarity>0:
        return 2
    elif analisis.polarity<0:
        return 0
    elif analisis.polarity==0:
        return 1

In [27]:
# Corro la función analisis_sentimiento que se creo sobe la columna review del data frame y lo reemplazo

df_user_reviews_sentimiento['review'] = df_user_reviews_sentimiento['review'].apply(analisis_sentimiento)


In [ ]:
df_user_reviews_sentimiento

In [29]:
# Renombrar la columna
df_user_reviews_sentimiento = df_user_reviews_sentimiento.rename(columns = {'review':'sentiment_analysis'})

In [ ]:
df_user_reviews_sentimiento

In [31]:
# Borro nuevamente los na

df_user_reviews_sentimiento = df_user_reviews_sentimiento.dropna()

In [ ]:
df_user_reviews_sentimiento

In [33]:
# Guardo el dataframe limpio con análisis de sentimiento

df_user_reviews_sentimiento.to_parquet('df_user_reviews.parquet')
